# TP 3 et 4 plan d'exécution de requêtes

date de modification: 01/02/2024

# Préparation

installer DuckDB


In [ ]:
# duckdb est déjà installé sur les machines de colab
# !pip install duckdb --pre

Les lib nécessaires pour ce TP

In [1]:
import duckdb

import sqlite3

import time
import pandas as pd

## Créer les données


On utilise la base de données du benchmark TPC-H.
DuckDB propose la procedure *dbgen* pour générer la base de données.
Les données sont stockées dans le fichier *tpch.db*


Comprendre le [schéma des données TPC-H](https://nuage.lip6.fr/s/RramHt5W3RomySs) (ou cf moodle).

La taille de la base est paramétrée par le scale factor *sf=0.2*


In [2]:
# db = duckdb.connect(':memory:')
db = duckdb.connect('tpch.db')
db.execute("CALL dbgen(sf=0.1)")

## Consulter la BD

Il y a 8 tables. La commande `SHOW TABLES` retourne le nom des tables.

Note: la fonction  `.df()` ajoutée à la fin de l'expression sert à importer le résultat de la requête dans un dataframe pandas afin d'avoir un rendu "ergonomique" dans colab .

In [4]:
db.execute("SHOW TABLES").df()

,name
0,customer
1,lineitem
2,nation
3,orders
4,part
5,partsupp
6,region
7,supplier


La commande `DESCRIBE` retourne une description des attributs d'une table.

Exemple affichant les attributs de la table *Lineitem* :

In [4]:
db.execute("DESCRIBE lineitem").df()

,column_name,column_type,null,key,default,extra
0,l_orderkey,INTEGER,NO,None,None,None
1,l_partkey,INTEGER,NO,None,None,None
2,l_suppkey,INTEGER,NO,None,None,None
3,l_linenumber,INTEGER,NO,None,None,None
4,l_quantity,"DECIMAL(15,2)",NO,None,None,None
5,l_extendedprice,"DECIMAL(15,2)",NO,None,None,None
6,l_discount,"DECIMAL(15,2)",NO,None,None,None
7,l_tax,"DECIMAL(15,2)",NO,None,None,None
8,l_returnflag,VARCHAR,NO,None,None,None
9,l_linestatus,VARCHAR,NO,None,None,None


La clause  `LIMIT N` est ajoutée à la fin d'une requête pour calculer seulement les N premiers tuples du résultat.

Exemple pour afficher 10 tuples de la table Lineitem :

In [5]:
db.execute("SELECT * FROM lineitem LIMIT 10").df()

,l_orderkey,l_partkey,l_suppkey,l_linenumber,l_quantity,l_extendedprice,l_discount,l_tax,l_returnflag,l_linestatus,l_shipdate,l_commitdate,l_receiptdate,l_shipinstruct,l_shipmode,l_comment
0,1,15519,785,1,17.0,24386.67,0.04,0.02,N,O,1996-03-13,1996-02-12,1996-03-22,DELIVER IN PERSON,TRUCK,to beans x-ray carefull
1,1,6731,732,2,36.0,58958.28,0.09,0.06,N,O,1996-04-12,1996-02-28,1996-04-20,TAKE BACK RETURN,MAIL,according to the final foxes. qui
2,1,6370,371,3,8.0,10210.96,0.10,0.02,N,O,1996-01-29,1996-03-05,1996-01-31,TAKE BACK RETURN,REG AIR,ourts cajole above the furiou
3,1,214,465,4,28.0,31197.88,0.09,0.06,N,O,1996-04-21,1996-03-30,1996-05-16,NONE,AIR,s cajole busily above t
4,1,2403,160,5,24.0,31329.60,0.10,0.04,N,O,1996-03-30,1996-03-14,1996-04-01,NONE,FOB,"the regular, regular pa"
5,1,1564,67,6,32.0,46897.92,0.07,0.02,N,O,1996-01-30,1996-02-07,1996-02-03,DELIVER IN PERSON,MAIL,rouches. special
6,2,10617,138,1,38.0,58049.18,0.00,0.05,N,O,1997-01-28,1997-01-14,1997-02-02,TAKE BACK RETURN,RAIL,re. enticingly regular instruct
7,3,430,181,1,45.0,59869.35,0.06,0.00,R,F,1994-02-02,1994-01-04,1994-02-23,NONE,AIR,s cajole above the pinto beans. iro
8,3,1904,658,2,49.0,88489.10,0.10,0.00,R,F,1993-11-09,1993-12-20,1993-11-24,TAKE BACK RETURN,RAIL,ecial pinto beans. sly
9,3,12845,370,3,27.0,47461.68,0.06,0.07,A,F,1994-01-16,1993-11-22,1994-01-23,DELIVER IN PERSON,SHIP,e carefully fina


La commande `SUMMARIZE` donne un aperçu statistique des données d'une table.
Elle retourne pour chaque attribut, les bornes min et max du domaine, le nombre approximatif de valeurs uniques ainsi que la distribution décrite par la moyenne, l'écart type et les quartiles 25, 50 (médiane) et 75.

In [6]:
db.execute("SUMMARIZE lineitem").df()

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,l_orderkey,INTEGER,1,600000,156886,300087.32032628893,173380.15954532547,149724,300442,450506,600572,0.0
1,l_partkey,INTEGER,1,20000,19915,10003.995747387491,5768.962517282627,5010,9995,15005,600572,0.0
2,l_suppkey,INTEGER,1,1000,1010,500.55533391500103,288.555762611197,251,501,749,600572,0.0
3,l_linenumber,INTEGER,1,7,7,3.001215507882485,1.73211719261282,1,3,4,600572,0.0
4,l_quantity,"DECIMAL(15,2)",1.00,50.00,50,25.53366124294839,14.420188609142432,13,25,38,600572,0.0
5,l_extendedprice,"DECIMAL(15,2)",901.00,95949.50,130790,35992.236201887536,22014.18174976403,17602,34467,51705,600572,0.0
6,l_discount,"DECIMAL(15,2)",0.00,0.10,11,0.050073929520523766,0.03160312819105228,0,0,0,600572,0.0
7,l_tax,"DECIMAL(15,2)",0.00,0.08,9,0.040041626982276896,0.02579730471367536,0,0,0,600572,0.0
8,l_returnflag,VARCHAR,A,R,3,None,None,None,None,None,600572,0.0
9,l_linestatus,VARCHAR,F,O,2,None,None,None,None,None,600572,0.0


In [7]:
db.execute("SUMMARIZE customer").df()

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,c_custkey,INTEGER,1,15000,15027,7500.5,4330.271354083945,3750,7500,11250,15000,0.0
1,c_name,VARCHAR,Customer#000000001,Customer#000015000,15232,None,None,None,None,None,15000,0.0
2,c_address,VARCHAR,3SUEwTaEs35ZDr5dx6P2b,zzyQcZpC50YDkK26Pglp,14922,None,None,None,None,None,15000,0.0
3,c_nationkey,INTEGER,0,24,25,11.9422,7.19689137820633,6,12,18,15000,0.0
4,c_phone,VARCHAR,10-103-318-6809,34-999-121-7718,14909,None,None,None,None,None,15000,0.0
5,c_acctbal,"DECIMAL(15,2)",-999.95,9999.72,14814,4470.497594,3175.721006098077,1746,4409,7230,15000,0.0
6,c_mktsegment,VARCHAR,AUTOMOBILE,MACHINERY,5,None,None,None,None,None,15000,0.0
7,c_comment,VARCHAR,"Tiresias haggle furiously bold, express instr...",zzle furiously regular foxes:,14835,None,None,None,None,None,15000,0.0


In [8]:
db.execute("SUMMARIZE orders").df()

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,o_orderkey,INTEGER,1,600000,156886,299991.5,173205.65812136806,149992,299991,449992,150000,0.0
1,o_custkey,INTEGER,1,14999,10024,7495.456166666667,4326.570605441797,3740,7485,11245,150000,0.0
2,o_orderstatus,VARCHAR,F,P,3,None,None,None,None,None,150000,0.0
3,o_totalprice,"DECIMAL(15,2)",833.40,479129.21,150104,142377.30687086665,83471.32571666414,73275,136097,202681,150000,0.0
4,o_orderdate,DATE,1992-01-01,1998-08-02,2404,None,None,None,None,None,150000,0.0
5,o_orderpriority,VARCHAR,1-URGENT,5-LOW,5,None,None,None,None,None,150000,0.0
6,o_clerk,VARCHAR,Clerk#000000001,Clerk#000001000,1012,None,None,None,None,None,150000,0.0
7,o_shippriority,INTEGER,0,0,1,0.0,0.0,0,0,0,150000,0.0
8,o_comment,VARCHAR,Tiresias affix after the silent courts,zzle: slyly even ideas wake furiously across t...,149436,None,None,None,None,None,150000,0.0


# Requêtes

On considère les requêtes suivantes qui sont des versions simplifiées d'une des requêtes du benchmark TPC-H :

In [5]:
query1 = """
SELECT
    o_shippriority, count(*) as nb
FROM
    customer
    JOIN orders ON (c_custkey=o_custkey)
    JOIN lineitem ON (l_orderkey=o_orderkey)
WHERE
    c_mktsegment = 'BUILDING'
    AND o_orderdate < CAST('1995-03-15' AS date)
    AND l_shipdate > CAST('1995-03-15' AS date)
GROUP BY
    o_shippriority
ORDER BY nb desc
LIMIT 50;
"""


query2 = """
SELECT
    l_orderkey,
    sum(l_extendedprice * (1 - l_discount)) AS revenue,
    o_orderdate,
    o_shippriority
FROM
    customer
    JOIN orders ON (c_custkey=o_custkey)
    JOIN lineitem ON (l_orderkey=o_orderkey)
WHERE
    c_mktsegment = 'BUILDING'
    AND o_orderdate < CAST('1995-03-15' AS date)
    AND l_shipdate > CAST('1995-03-15' AS date)
GROUP BY
    l_orderkey,
    o_orderdate,
    o_shippriority
ORDER BY
    revenue DESC,
    o_orderdate
LIMIT 50;
"""

query = query1

## Exécuter une requête

On définit la fonction `run_query` pour exécuter une requête et mesurer sa durée en millisecondes.

In [6]:
# par défaut on exécute la requête dans DuckDB (db)
def run_query(q, connection=db):
  start = time.time()
  connection.execute(q).fetchall()
  end = time.time()
  return(f"{(end - start)*1000:.0f}ms")


Exécuter la première requête dans DuckDB

In [10]:
run_query(query)

'29ms'

# Comparaison DuckDB versus SQLite

L'objectif est de comparer les performances de deux SGBD en mémoire :
DuckDB (très rapide) et SQLite (très populaire)

On veut montrer que DuckDB est plus rapide que SQLite pour traiter des requêtes "réalistes" de l'application TPC-H.


## Configuration de SQLite

Initialiser SQLite.

In [7]:
# sqlite = sqlite3.connect(':memory:', check_same_thread=False)
!rm -f tpch_sqlite.db
sqlite = sqlite3.connect('tpch_sqlite.db', check_same_thread=False)

On peut transférer les données depuis DuckDB vers SQLite par l'intermédaire d'un dataFrame pandas.
*   On exporte chaque table de DuckDB vers un dataframe en utilisant  la commande `.df()`
*   Puis on utilise la fonction `to_sql` d'un dataframe pandas pour générer les instructions SQL permettant d'importer les données dans SQLite.


In [8]:
table_list = db.execute("SHOW TABLES").fetchall()

# peut durer de 20s à  1 minute

for table in table_list:
  tname = table[0]
  table_data = db.table(tname).df()
  table_data.to_sql(tname, sqlite)
  # table_data.to_sql(tname, sqlite, if_exists='replace')


## Comparer les performances

On exécute la même requête dans les 2 SGBD: DuckDB et SQLite. Puis on compare les temps d'exécution.

In [9]:
print("On exécute la requête:", query)

duckdb_result = run_query(query, db)
sqlite_result = run_query(query, sqlite)

print("Les durées d'exécution sont: ")

pd.DataFrame.from_dict({
    'DuckDB': [duckdb_result],
    'SQLite': [sqlite_result]
})

On exécute la requête: 
SELECT
    o_shippriority, count(*) as nb
FROM
    customer
    JOIN orders ON (c_custkey=o_custkey)
    JOIN lineitem ON (l_orderkey=o_orderkey)
WHERE
    c_mktsegment = 'BUILDING'
    AND o_orderdate < CAST('1995-03-15' AS date)
    AND l_shipdate > CAST('1995-03-15' AS date)
GROUP BY
    o_shippriority
ORDER BY nb desc
LIMIT 50;



Les durées d'exécution sont: 


,DuckDB,SQLite
0,19ms,1284ms


On constate que DuckDB est environ 20 fois plus rapide que SQLite pour cette requête.

Afin de montrer l'efficacité de l'optimiseur de requêtes,
on demande à DuckDB d'exécuter la requête sans l'optimiser.

La commande  `PRAGMA disable_optimizer` désactive l'optimiseur de requête.
Puis on re-exécute la requête pour mesurer sa durée.

In [10]:
db.execute("PRAGMA disable_optimizer")
duckdb_unoptimized_result = run_query(query)
db.execute("PRAGMA enable_optimizer")

pd.DataFrame.from_dict({
    'DuckDB': [duckdb_result],
    'DuckDB (sans optimisation de requête)': [duckdb_unoptimized_result],
    'SQLite': [sqlite_result]
})

,DuckDB,DuckDB (sans optimisation de requête),SQLite
0,19ms,193ms,1284ms


On constate que la requête sans optimisation s'exécute beaucoup plus lentement (durée presque 5 fois plus longue!) que celle optimisée.

# Plan d'une requête
L'écart de durée entre la requête optimisée et celle non optimisée vient du fait qu'elles ont des plans d'exécution différents

On peut visualiser le **plan** d'exécution d'une requête en préfixant la requête par `EXPLAIN`.

La commande `EXPLAIN` ne fait qu'afficher le plan d'exécution ; elle n'exécute **pas** la requête.

Le plan affiché est appelé *plan physique* car il indique pour chaque opérateur du plan, le nom de l'algorithme utilisé:
*   HASH_GROUP_BY : regroupement par hachage puis agrégation
*   HASH_JOIN : jointure par hachage
*   FILTER : sélection en pipeline
*   SEQ_SCAN : lecture séquentielle d'une table

De plus le plan indique la cardinalité estimée **EC** pour certains opérateurs, en particulier pour les sélections et les jointures.

In [11]:
def explain_query(query):
  print(db.execute("EXPLAIN " + query).fetchall()[0][1])

print(query)
explain_query(query)


SELECT
    o_shippriority, count(*) as nb
FROM
    customer
    JOIN orders ON (c_custkey=o_custkey)
    JOIN lineitem ON (l_orderkey=o_orderkey)
WHERE
    c_mktsegment = 'BUILDING'
    AND o_orderdate < CAST('1995-03-15' AS date)
    AND l_shipdate > CAST('1995-03-15' AS date)
GROUP BY
    o_shippriority
ORDER BY nb desc
LIMIT 50;

┌───────────────────────────┐                                                          
│           TOP_N           │                                                          
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                                                          
│           Top 50          │                                                          
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                                                          
│     count_star() DESC     │                                                          
└─────────────┬─────────────┘                                                                                       
┌─────────────┴────

Remarque : vous pourriez invoquer `PRAGMA explain_output='all'` pour afficher, en plus du plan physique, le plan logique avant son optimisation et le plan logique optimisé, mais ce n'est pas demandé dans ce TP.

# Rapport d'exécution d'un plan

On peut afficher un rapport **après** l'exécution d'un plan pour connaitre la quantité de données traitées par chaque opération du plan. On appelle cela le *profil* de l'exécution.

La fonction  `run_and_profile_query` active le mode profil afin de sauvegarder les informations de profil dans un fichier.
La requête est ensuite exécutée puis les informations de profil produites pendant l'exécution sont affichées.

On peut lire dans le profil, pour chaque opérateur, la durée écoulée, le nombre de tuples consommés en entrée ainsi que le nombre de tuples produits pour l'opérateur suivant.

Plus spécifiquement, l'opérateur de lecture séquentielle `SEQ_SCAN` (sequential scan) indique le nombre de tuples lus dans une table.
L'opérateur `HASH_GROUP_BY` indique le nombre de groupes créés.

Remarque: pour les opérateurs qui matérialisent leur sortie, le nombre de tuples produits représente la cardinalité du résultat intermédiaire.

Les cardinalités intermédiaires sont importantes car elles peuvent expliquer pourquoi un opérateur a pris beaucoup de temps pour être traité.
Dans de nombreuses situations, il est possible de réduire la cardinalité des résultats intermédiaires et modifiant l'ordre des opérations et la façon dont elles sont traitées.


In [12]:
def run_and_profile_query(query):
  db.execute("PRAGMA enable_profiling")
  db.execute("PRAGMA profiling_output='out.log'")
  db.execute(query)
  db.execute("PRAGMA disable_profiling")
  with open('out.log', 'r') as f:
    output = f.read()
  print(output)

run_and_profile_query(query)

┌─────────────────────────────────────┐
│┌───────────────────────────────────┐│
││    Query Profiling Information    ││
│└───────────────────────────────────┘│
└─────────────────────────────────────┘
 SELECT     o_shippriority, count(*) as nb FROM     customer     JOIN orders ON (c_custkey=o_custkey)     JOIN lineitem ON (l_orderkey=o_orderkey) WHERE     c_mktsegment = 'BUILDING'     AND o_orderdate < CAST('1995-03-15' AS date)     AND l_shipdate > CAST('1995-03-15' AS date) GROUP BY     o_shippriority ORDER BY nb desc LIMIT 50; 
┌─────────────────────────────────────┐
│┌───────────────────────────────────┐│
││        Total Time: 0.0544s        ││
│└───────────────────────────────────┘│
└─────────────────────────────────────┘
┌───────────────────────────┐                                                          
│           TOP_N           │                                                          
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                                                          



# Exercice 1 : Optimiser un première requête

Pour mieux comprendre le fonctionnement de l'optimiseur de requêtes, on propose d'optimiser la requête "à la main" en SQL.
Pour cela, on désactive l'optimiseur de requêtes avec la commande `PRAGMA disable_optimizer`. Cela signifie que le SGBD va traduire la requête SQL dans un plan qui sera "le plus proche possible" de l'ordre des insctructions écrites dans la requête SQL.


## Requête à optimiser

In [13]:
query1_cross_product = """
SELECT
    o_shippriority, count(*) as nb
FROM
    customer,
    orders,
    lineitem
WHERE
    c_custkey=o_custkey
    AND l_orderkey=o_orderkey
    AND c_mktsegment = 'BUILDING'
    AND o_orderdate < CAST('1995-03-15' AS date)
    AND l_shipdate > CAST('1995-03-15' AS date)
GROUP BY
    o_shippriority
ORDER BY nb desc
LIMIT 50;
"""

# query2_cross_product = """
# SELECT
#     l_orderkey,
#     sum(l_extendedprice * (1 - l_discount)) AS revenue,
#     o_orderdate,
#     o_shippriority
# FROM
#     customer,
#     orders,
#     lineitem
# WHERE
#     c_custkey=o_custkey
#     AND l_orderkey=o_orderkey
#     AND c_mktsegment = 'BUILDING'
#     AND o_orderdate < CAST('1995-03-15' AS date)
#     AND l_shipdate > CAST('1995-03-15' AS date)
# GROUP BY
#     l_orderkey,
#     o_orderdate,
#     o_shippriority
# ORDER BY
#     revenue DESC,
#     o_orderdate
# LIMIT 50;
# """

Afficher le plan de la requête

In [14]:
db.execute("PRAGMA disable_optimizer")
explain_query(query1_cross_product)

# Note: ne pas executer cette requête car son plan est tres lent!

┌───────────────────────────┐                                                          
│           LIMIT           │                                                          
└─────────────┬─────────────┘                                                                                       
┌─────────────┴─────────────┐                                                          
│          ORDER_BY         │                                                          
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                                                          
│          ORDERS:          │                                                          
│     count_star() DESC     │                                                          
└─────────────┬─────────────┘                                                                                       
┌─────────────┴─────────────┐                                                          
│       HASH_GROUP_BY       │                                 

On voit que le plan ne contient aucune jointure mais des produits cartésiens suivis de sélections. Or les produits cartésiens sont coûteux à calculer!

On évite d'exécuter cette requête pendant le TP car cela  prendrait trop de temps...



On remplace les produits cartésiens par des jointures en ajoutant  des `JOIN` dans la clause `FROM` (comme dans query1).

Cela rend la requête plus rapide et on peut l'exécuter pour mesurer sa durée.

In [20]:
query_join = """
SELECT
    o_shippriority, count(*) as nb
FROM
    customer
    JOIN orders ON (c_custkey=o_custkey)
    JOIN lineitem ON (l_orderkey=o_orderkey)
WHERE
    c_mktsegment = 'BUILDING'
    AND o_orderdate < CAST('1995-03-15' AS date)
    AND l_shipdate > CAST('1995-03-15' AS date)
GROUP BY
    o_shippriority
ORDER BY nb desc
LIMIT 50;
"""

explain_query(query_join)
run_query(query_join)

┌───────────────────────────┐                                                          
│           LIMIT           │                                                          
└─────────────┬─────────────┘                                                                                       
┌─────────────┴─────────────┐                                                          
│          ORDER_BY         │                                                          
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                                                          
│          ORDERS:          │                                                          
│     count_star() DESC     │                                                          
└─────────────┬─────────────┘                                                                                       
┌─────────────┴─────────────┐                                                          
│       HASH_GROUP_BY       │                                 

'221ms'

## a) Traiter les sélections avant les jointures

Prroposer d'autres écritures SQL équivalentes de la requête pour qu'elle soit traitée encore plus rapidement.

Utiliser des requêtes imbriquées pour pousser (ou non) les sélections et les projections.

Vérifier que la requête proposées est bien équivalente à la requête initiale
(avoir le même résultat est une condition nécessaire).



In [21]:
query_a = """
SELECT
    o_shippriority, count(*) as nb
FROM
    (SELECT c_custkey FROM customer WHERE c_mktsegment = 'BUILDING')
    JOIN (SELECT o_orderkey ,o_custkey , o_shippriority FROM orders WHERE o_orderdate < CAST('1995-03-15' AS date))  ON (c_custkey=o_custkey)
    JOIN (SELECT l_orderkey , l_shipdate  FROM lineitem  WHERE l_shipdate > CAST('1995-03-15' AS date) ) ON (l_orderkey=o_orderkey)

GROUP BY
    o_shippriority
ORDER BY nb desc
LIMIT 50;
"""

explain_query(query_a)
run_query(query_a)

┌───────────────────────────┐                                                          
│           LIMIT           │                                                          
└─────────────┬─────────────┘                                                                                       
┌─────────────┴─────────────┐                                                          
│          ORDER_BY         │                                                          
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                                                          
│          ORDERS:          │                                                          
│     count_star() DESC     │                                                          
└─────────────┬─────────────┘                                                                                       
┌─────────────┴─────────────┐                                                          
│       HASH_GROUP_BY       │                                 

'50ms'

## b) Ordre des jointures

On peut voir que les tables sont jointes dans l'ordre Order,Customer, Lineitem

Re-ecrire la requête pour que les tables ne soient pas jointes dans des ordres différents. Est ce que cela change la durée de la requête ?

In [22]:
query_b = """
SELECT
    o_shippriority, count(*) as nb
FROM
    (SELECT l_orderkey , l_shipdate  FROM lineitem  WHERE l_shipdate > CAST('1995-03-15' AS date) )
    JOIN (SELECT o_orderkey ,o_custkey , o_shippriority FROM orders WHERE o_orderdate < CAST('1995-03-15' AS date)) ON (l_orderkey=o_orderkey)
    JOIN (SELECT c_custkey FROM customer WHERE c_mktsegment = 'BUILDING') ON (c_custkey=o_custkey)

GROUP BY
    o_shippriority
ORDER BY nb desc
LIMIT 50;
"""
#charcher une requete qui montre que ce temps d'amélioration est grand
explain_query(query_b)
run_query(query_b)

┌───────────────────────────┐                                                          
│           LIMIT           │                                                          
└─────────────┬─────────────┘                                                                                       
┌─────────────┴─────────────┐                                                          
│          ORDER_BY         │                                                          
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                                                          
│          ORDERS:          │                                                          
│     count_star() DESC     │                                                          
└─────────────┬─────────────┘                                                                                       
┌─────────────┴─────────────┐                                                          
│       HASH_GROUP_BY       │                                 

'31ms'

## c) Index

Créer des index et montrer si cela a un effet sur la durée de la requête.

### Activation de l'optimiseur
On remarque pour cette requete lorsqu'on active l'optimizeur l'index est utilisé

In [43]:

db.execute("PRAGMA enable_optimizer")
db.execute("CREATE INDEX o_orderkey_idx ON orders (o_orderkey)")


query_c = """
    SELECT
        o_orderkey
    FROM
        orders 
    WHERE 
        o_orderkey = 500
"""

# Exécuter la requête
explain_query(query_c)
run_query(query_c)


┌───────────────────────────┐
│           FILTER          │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│     (o_orderkey = 500)    │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│         EC: 30000         │
└─────────────┬─────────────┘                             
┌─────────────┴─────────────┐
│        INDEX_SCAN         │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│           orders          │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│         o_orderkey        │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│         EC: 150000        │
└───────────────────────────┘                             



'1ms'

### Désactivation de l'optimiSeur
On remarque pour cette requete lorsqu'on désactive l'optimizeur l'index n'est pas utilisé

In [44]:
db.execute("PRAGMA disable_optimizer")
query_c = """
    SELECT
        o_orderkey
    FROM
        orders 
    WHERE 
        o_orderkey = 500
"""

# Exécuter la requête
explain_query(query_c)
run_query(query_c)

┌───────────────────────────┐
│           FILTER          │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│ (o_orderkey = CAST(500 AS │
│          INTEGER))        │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│         EC: 150000        │
└─────────────┬─────────────┘                             
┌─────────────┴─────────────┐
│         SEQ_SCAN          │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│           orders          │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│         EC: 150000        │
└───────────────────────────┘                             



'4ms'

In [45]:
db.execute("DROP INDEX o_orderkey_idx")

### TEST sur un attribut unique mais sur une requête avec une opération autre qu'une = : 

In [ ]:
# db.execute("DROP INDEX l_orderkey_index ")
db.execute("CREATE INDEX o_orderkey_index ON orders (o_orderkey)")



In [ ]:
db.execute("PRAGMA enable_optimizer")

query_index1 = """
SELECT
    o_orderkey
FROM
  orders
WHERE
  o_orderkey > 1
"""
explain_query(query_index1)
run_query(query_index1)
# run_and_profile_query(query_index1)

┌───────────────────────────┐
│         SEQ_SCAN          │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│           orders          │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│         o_orderkey        │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│ Filters: o_orderkey>1 AND │
│   o_orderkey IS NOT NULL  │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│         EC: 30000         │
└───────────────────────────┘                             



'46ms'

In [ ]:
db.execute("PRAGMA disable_optimizer")
explain_query(query_index1)
run_query(query_index1)
# run_and_profile_query(query_index)

┌───────────────────────────┐
│           FILTER          │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│      (o_orderkey > 1)     │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│         EC: 150000        │
└─────────────┬─────────────┘                             
┌─────────────┴─────────────┐
│         SEQ_SCAN          │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│           orders          │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│         EC: 150000        │
└───────────────────────────┘                             



'44ms'

On voit bien que malgré le fait que l'optimiseur soit activé l'index n'a pas été utilisé , et cela parait assez logique car sinon pour chaque valeur on devra refaire une nouvelle lecture (assez couteux)

### Jointure avec Index et sans

In [48]:
db.execute("PRAGMA enable_optimizer")

db.execute("CREATE INDEX l_orderkey_idx ON lineitem (l_orderkey)")


query_d = """
    SELECT o_orderkey , l_orderkey
    FROM orders 
         JOIN lineitem ON (o_orderkey = l_orderkey)
    WHERE l_orderkey >= 1 and l_orderkey <= 1000
"""

# Exécuter la requête
explain_query(query_d)
run_query(query_d)

┌───────────────────────────┐                             
│         PROJECTION        │                             
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
│         o_orderkey        │                             
│         l_orderkey        │                             
└─────────────┬─────────────┘                                                          
┌─────────────┴─────────────┐                             
│         HASH_JOIN         │                             
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
│           INNER           │                             
│  l_orderkey = o_orderkey  ├──────────────┐              
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │              │              
│         EC: 24414         │              │              
└─────────────┬─────────────┘              │                                           
┌─────────────┴─────────────┐┌─────────────┴─────────────┐
│           FILTER          ││         SEQ_SCAN          

'7ms'

In [49]:
db.execute("PRAGMA disable_optimizer")

# Exécuter la requête
explain_query(query_d)
run_query(query_d)

┌───────────────────────────┐                             
│           FILTER          │                             
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
│ ((l_orderkey >= CAST(1 AS │                             
│ INTEGER)) AND (l_orde...  │                             
│   CAST(1000 AS INTEGER))) │                             
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
│         EC: 600572        │                             
└─────────────┬─────────────┘                                                          
┌─────────────┴─────────────┐                             
│         HASH_JOIN         │                             
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
│           INNER           │                             
│  o_orderkey = l_orderkey  ├──────────────┐              
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │              │              
│         EC: 600572        │              │              
└─────────────┬────────────

'139ms'

In [74]:
db.execute("DROP INDEX l_orderkey_idx")

### Selection sur c_mktsegment
Qui est non unique : 

In [63]:
db.execute("DROP INDEX c_mktsegment_idx")

In [64]:
db.execute("PRAGMA enable_optimizer")
db.execute("CREATE INDEX c_mktsegment_idx ON customer (c_mktsegment)")


query_e = """
    SELECT c_custkey
    FROM customer 
    WHERE c_mktsegment = 'AUTOMOBILE'
"""

# Exécuter la requête
explain_query(query_e)
run_query(query_e)

┌───────────────────────────┐
│         SEQ_SCAN          │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│          customer         │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│         c_custkey         │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│   Filters: c_mktsegment   │
│=AUTOMOBILE AND c_mkts...  │
│          NOT NULL         │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│          EC: 3000         │
└───────────────────────────┘                             



'3ms'

In [66]:
db.execute("PRAGMA disable_optimizer")

# Exécuter la requête
explain_query(query_e)
run_query(query_e)

┌───────────────────────────┐
│         PROJECTION        │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│         c_custkey         │
└─────────────┬─────────────┘                             
┌─────────────┴─────────────┐
│           FILTER          │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│   (c_mktsegment = CAST(   │
│ 'AUTOMOBILE' AS VARCHAR)) │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│         EC: 15000         │
└─────────────┬─────────────┘                             
┌─────────────┴─────────────┐
│         SEQ_SCAN          │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│          customer         │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│         EC: 15000         │
└───────────────────────────┘                             



'3ms'

L'optimiseur n'a donc pas utilsier l index car c'est moins optimale de l'utiliser

# Exercice 2 : Optimiser les requêtes TPCH

Optimiser chaque requête de TPC-H.
Vous pouvez les obtenir avec `SELECT * FROM tpch_queries()`


In [18]:
queries = db.execute("SELECT * FROM tpch_queries()").fetchall()

# for id, sql in queries:
for id, sql in queries[:5]:
  print('### requete ', id, ':')
  print(sql)

### requete  1 :
SELECT
    l_returnflag,
    l_linestatus,
    sum(l_quantity) AS sum_qty,
    sum(l_extendedprice) AS sum_base_price,
    sum(l_extendedprice * (1 - l_discount)) AS sum_disc_price,
    sum(l_extendedprice * (1 - l_discount) * (1 + l_tax)) AS sum_charge,
    avg(l_quantity) AS avg_qty,
    avg(l_extendedprice) AS avg_price,
    avg(l_discount) AS avg_disc,
    count(*) AS count_order
FROM
    lineitem
WHERE
    l_shipdate <= CAST('1998-09-02' AS date)
GROUP BY
    l_returnflag,
    l_linestatus
ORDER BY
    l_returnflag,
    l_linestatus;

### requete  2 :
SELECT
    s_acctbal,
    s_name,
    n_name,
    p_partkey,
    p_mfgr,
    s_address,
    s_phone,
    s_comment
FROM
    part,
    supplier,
    partsupp,
    nation,
    region
WHERE
    p_partkey = ps_partkey
    AND s_suppkey = ps_suppkey
    AND p_size = 15
    AND p_type LIKE '%BRASS'
    AND s_nationkey = n_nationkey
    AND n_regionkey = r_regionkey
    AND r_name = 'EUROPE'
    AND ps_supplycost = (
      

## Requête 1 :

In [40]:
query_1_tpch =  queries[0][1]
print(query_1_tpch)

SELECT
    l_returnflag,
    l_linestatus,
    sum(l_quantity) AS sum_qty,
    sum(l_extendedprice) AS sum_base_price,
    sum(l_extendedprice * (1 - l_discount)) AS sum_disc_price,
    sum(l_extendedprice * (1 - l_discount) * (1 + l_tax)) AS sum_charge,
    avg(l_quantity) AS avg_qty,
    avg(l_extendedprice) AS avg_price,
    avg(l_discount) AS avg_disc,
    count(*) AS count_order
FROM
    lineitem
WHERE
    l_shipdate <= CAST('1998-09-02' AS date)
GROUP BY
    l_returnflag,
    l_linestatus
ORDER BY
    l_returnflag,
    l_linestatus;



In [67]:
db.execute("PRAGMA enable_optimizer")
# Exécuter la requête
explain_query(query_1_tpch)
run_query(query_1_tpch)

┌───────────────────────────┐
│         PROJECTION        │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│__internal_decompress_strin│
│           g(#0)           │
│__internal_decompress_strin│
│           g(#1)           │
│             #2            │
│             #3            │
│             #4            │
│             #5            │
│             #6            │
│             #7            │
│             #8            │
│             #9            │
└─────────────┬─────────────┘                             
┌─────────────┴─────────────┐
│          ORDER_BY         │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│          ORDERS:          │
│ lineitem.l_returnflag ASC │
│ lineitem.l_linestatus ASC │
└─────────────┬─────────────┘                             
┌─────────────┴─────────────┐
│         PROJECTION        │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│__internal_compress_string_│
│        utinyint(#0)       │
│__internal_compress_string_│
│        utinyint(#1)       │
│             #2            │
│           

'60ms'

In [69]:
db.execute("CREATE INDEX l_shipdate_idx ON lineitem (l_shipdate)")
# Exécuter la requête
explain_query(query_1_tpch)
run_query(query_1_tpch)

┌───────────────────────────┐
│         PROJECTION        │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│__internal_decompress_strin│
│           g(#0)           │
│__internal_decompress_strin│
│           g(#1)           │
│             #2            │
│             #3            │
│             #4            │
│             #5            │
│             #6            │
│             #7            │
│             #8            │
│             #9            │
└─────────────┬─────────────┘                             
┌─────────────┴─────────────┐
│          ORDER_BY         │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│          ORDERS:          │
│ lineitem.l_returnflag ASC │
│ lineitem.l_linestatus ASC │
└─────────────┬─────────────┘                             
┌─────────────┴─────────────┐
│         PROJECTION        │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│__internal_compress_string_│
│        utinyint(#0)       │
│__internal_compress_string_│
│        utinyint(#1)       │
│             #2            │
│           

'31ms'

In [70]:
db.execute("DROP INDEX l_shipdate_idx")

## Requête 3 : 

In [ ]:
#Traitement de la requete 3 :
query_3_to_optimize = """
SELECT
    l_orderkey,
    sum(l_extendedprice * (1 - l_discount)) AS revenue,
    o_orderdate,
    o_shippriority
FROM
    customer,
    orders,
    lineitem
WHERE
    c_mktsegment = 'BUILDING'
    AND c_custkey = o_custkey
    AND l_orderkey = o_orderkey
    AND o_orderdate < CAST('1995-03-15' AS date)
    AND l_shipdate > CAST('1995-03-15' AS date)
GROUP BY
    l_orderkey,
    o_orderdate,
    o_shippriority
ORDER BY
    revenue DESC,
    o_orderdate
LIMIT 10;
"""

In [ ]:
db.execute("PRAGMA enable_optimizer")
explain_query(query_3_to_optimize)
run_query(query_3_to_optimize)

┌───────────────────────────┐                                                          
│           TOP_N           │                                                          
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                                                          
│           Top 10          │                                                          
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                                                          
│       sum((lineitem       │                                                          
│.l_extendedprice * (1 ...  │                                                          
│    .l_discount))) DESC    │                                                          
│   orders.o_orderdate ASC  │                                                          
└─────────────┬─────────────┘                                                                                       
┌─────────────┴─────────────┐                                                          
│  

'23ms'

Ici le temps d'execution est enorme on essaye de traiter les selection et projection avant les jointures

In [ ]:
#Traitement de la requete 3 :
db.execute("PRAGMA disable_optimizer")

query_3_optimized_v1 = """
SELECT
    l_orderkey,
    sum(l_extendedprice * (1 - l_discount)) AS revenue,
    o_orderdate,
    o_shippriority
FROM
    (select c_custkey from customer where c_mktsegment = 'BUILDING')
    JOIN ( select o_orderdate,o_shippriority,o_custkey,o_orderkey from orders WHERE o_orderdate < CAST('1995-03-15' AS date)) ON (o_custkey = c_custkey )
    JOIN (select  l_orderkey,l_extendedprice,l_discount from lineitem WHERE l_shipdate > CAST('1995-03-15' AS date)) ON (l_orderkey = o_orderkey  )
GROUP BY
    l_orderkey,
    o_orderdate,
    o_shippriority
ORDER BY
    revenue DESC,
    o_orderdate
LIMIT 10;
"""
explain_query(query_3_optimized_v1)
run_query(query_3_optimized_v1)

┌───────────────────────────┐                                                          
│           LIMIT           │                                                          
└─────────────┬─────────────┘                                                                                       
┌─────────────┴─────────────┐                                                          
│          ORDER_BY         │                                                          
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                                                          
│          ORDERS:          │                                                          
│  sum((unnamed_subquery15  │                                                          
│  .l_extendedprice * (1 -  │                                                          
│ unnamed_subquery15.l_...  │                                                          
│            DESC           │                                                          
│  

'43ms'

Meilleure mais on peut faire mieux (voir la suite)

In [ ]:
#Traitement de la requete 3 :
db.execute("PRAGMA disable_optimizer")

query_3_optimized_v2 = """
SELECT
    l_orderkey,
    sum(l_extendedprice * (1 - l_discount)) AS revenue,
    o_orderdate,
    o_shippriority
FROM
     (select  l_orderkey,l_extendedprice,l_discount from lineitem WHERE l_shipdate > CAST('1995-03-15' AS date))
    JOIN (  ( select o_orderdate,o_shippriority,o_custkey,o_orderkey from orders WHERE o_orderdate < CAST('1995-03-15' AS date))
   JOIN  (select c_custkey from customer where c_mktsegment = 'BUILDING') ON (o_custkey = c_custkey ) )ON (l_orderkey = o_orderkey  )



GROUP BY
    l_orderkey,
    o_orderdate,
    o_shippriority
ORDER BY
    revenue DESC,
    o_orderdate
LIMIT 10;
"""
explain_query(query_3_optimized_v2)
run_query(query_3_optimized_v2)

┌───────────────────────────┐                                                          
│           LIMIT           │                                                          
└─────────────┬─────────────┘                                                                                       
┌─────────────┴─────────────┐                                                          
│          ORDER_BY         │                                                          
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                                                          
│          ORDERS:          │                                                          
│   sum((unnamed_subquery1  │                                                          
│  .l_extendedprice * (1 -  │                                                          
│ unnamed_subquery1.l_d...  │                                                          
│            DESC           │                                                          
│  

'19ms'

C'est la plus optimale et d'ailleur on remarque que c'est c'est celle qui est choisie lorsqu'on active l'optimiseur.

## Requête 4 :

In [71]:
query_4_tpch =  queries[3][1]
print(query_4_tpch)

SELECT
    o_orderpriority,
    count(*) AS order_count
FROM
    orders
WHERE
    o_orderdate >= CAST('1993-07-01' AS date)
    AND o_orderdate < CAST('1993-10-01' AS date)
    AND EXISTS (
        SELECT
            *
        FROM
            lineitem
        WHERE
            l_orderkey = o_orderkey
            AND l_commitdate < l_receiptdate)
GROUP BY
    o_orderpriority
ORDER BY
    o_orderpriority;



In [104]:
db.execute("PRAGMA enable_optimizer")
# Exécuter la requête
explain_query(query_4_tpch)
run_query(query_4_tpch)

┌───────────────────────────┐                                                                                       
│         PROJECTION        │                                                                                       
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                                                                                       
│__internal_decompress_strin│                                                                                       
│           g(#0)           │                                                                                       
│             #1            │                                                                                       
└─────────────┬─────────────┘                                                                                                                    
┌─────────────┴─────────────┐                                                                                       
│          ORDER_BY         │      

'32ms'

In [111]:
query_4_tpch_opt = """
SELECT
    o_orderpriority,
    count(*) AS order_count
FROM
    orders
    JOIN lineitem ON (o_orderkey = l_orderkey)
WHERE
    o_orderdate >= CAST('1993-07-01' AS date)
    AND o_orderdate < CAST('1993-10-01' AS date)
    AND l_commitdate < l_receiptdate
GROUP BY
    o_orderpriority
ORDER BY
    o_orderpriority;
"""
db.execute("PRAGMA enable_optimizer")
# Exécuter la requête
explain_query(query_4_tpch_opt)
run_query(query_4_tpch_opt)

┌───────────────────────────┐                             
│         PROJECTION        │                             
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
│__internal_decompress_strin│                             
│           g(#0)           │                             
│             #1            │                             
└─────────────┬─────────────┘                                                          
┌─────────────┴─────────────┐                             
│          ORDER_BY         │                             
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
│          ORDERS:          │                             
│ orders.o_orderpriority ASC│                             
└─────────────┬─────────────┘                                                          
┌─────────────┴─────────────┐                             
│         PROJECTION        │                             
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                            

'28ms'

In [121]:
db.execute("PRAGMA enable_optimizer")
db.execute("CREATE INDEX l_orderkey_idx ON lineitem (l_orderkey)")

# Exécuter la requête
explain_query(query_4_tpch_opt)
run_query(query_4_tpch_opt)

┌───────────────────────────┐                             
│         PROJECTION        │                             
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
│__internal_decompress_strin│                             
│           g(#0)           │                             
│             #1            │                             
└─────────────┬─────────────┘                                                          
┌─────────────┴─────────────┐                             
│          ORDER_BY         │                             
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
│          ORDERS:          │                             
│ orders.o_orderpriority ASC│                             
└─────────────┬─────────────┘                                                          
┌─────────────┴─────────────┐                             
│         PROJECTION        │                             
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                            

'19ms'

In [ ]:
db.execute("DROP INDEX l_orderkey_idx")

# Exercice 3 : Cardinalités

On considère le plan (optimisé) de la requête `query1`.

Expliquer comment est calculée chaque valeur de cardinalité estimée **EC**.

In [ ]:
db.execute("PRAGMA enable_optimizer")
explain_query(query1_cross_product)
run_query(query1_cross_product)


┌───────────────────────────┐                                                          
│           TOP_N           │                                                          
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                                                          
│           Top 50          │                                                          
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                                                          
│     count_star() DESC     │                                                          
└─────────────┬─────────────┘                                                                                       
┌─────────────┴─────────────┐                                                          
│         PROJECTION        │                                                          
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                                                          
│__internal_decompress_integ│                                                          
│  

'20ms'

Calculons la valeurs de cardinalité estimée :

1.   Pour ce qui concerne le filtrage de customer Filters: c_mktsegment = BUILDING :
  *   La table possède 15000 lignes au total .
  *   L'attribut c_mktsegment peut prendre 5 valeurs distinctes ( si l'on suppose que cela suit une distribution uniforme) le calcul de vient donc
      *   (1/5) * 15000
      *   D'où le EC = 3000
2.   Pour ce qui concerne le filtrage de orders Filters: o_orderdate < 1995-03-15 :
  *   La table possède 150000 lignes au total .
  *   L'attribut o_orderdate peut prendre 2404 valeurs distinctes :
      *   Entre date min est celle specifiée dans le filre nous avons près de 1169 dates.
      *   (1169 / 2404) * 150000
      *   D'où le EC = 72940 (ici nous n'avons pas trouvé le même résultat)
    
A noté : dans la suite de nos calculs, nous prendrons en considération les estimations de cardinalité fournies par l'optimiseur

3.   Pour ce qui concerne la jointure par hashage entre order et custemer :
  *   La table à droite de la jointure posséde 30000 lignes et celle à gauche posséde 3000 lignes
  *   Le facteur de selectivité est égale à (1/15000) car on a 15000 valeurs differentes de c_custkey
      *   d'où EC = (30000 * 3000 / 15000) = 6000 (ici nous n'avons pas trouvé le même résultat)

2.   Pour ce qui concerne le filtrage de lineitem Filters: l_shipdate >1995-03-15:
  *   La table possède 600572 lignes au total .
  *   L'attribut l_shipdate peut prendre 2515 valeurs distinctes :
      *   Entre la date max est celle specifiée dans le filre nous avons près de 1351 dates (jours).
      *   (1351 / 2515) * 600572
      *   D'où le EC = 322613 (ici nous n'avons pas trouvé le même résultat)

Et enfin apès le hash JOIN on a  : 30000 * 120114 * (1 / 150000) = 3086 (presque égale à 3136) .
Pour la clè de jointure on a une chance sur le nombre total de clès qui est égale à 150000.

# Exercice 4 : Optimiser une requête avec 6 tables

Soit la requête suivante qui accède à 6 tables.

In [19]:
query_6_tables = """
SELECT
    nation,
    o_year,
    sum(amount) AS sum_profit
FROM (
    SELECT
        n_name AS nation,
        extract(year FROM o_orderdate) AS o_year,
        l_extendedprice * (1 - l_discount) - ps_supplycost * l_quantity AS amount
    FROM
        part,
        supplier,
        lineitem,
        partsupp,
        orders,
        nation
    WHERE
        s_suppkey = l_suppkey
        AND ps_suppkey = l_suppkey
        AND ps_partkey = l_partkey
        AND p_partkey = l_partkey
        AND o_orderkey = l_orderkey
        AND s_nationkey = n_nationkey
        AND p_name LIKE '%green%') AS profit
GROUP BY
    nation,
    o_year
ORDER BY
    nation,
    o_year DESC;
"""
# Exécuter la requête
explain_query(query_6_tables)
run_query(query_6_tables)

┌───────────────────────────┐                                                                                                                                                 
│         PROJECTION        │                                                                                                                                                 
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                                                                                                                                                 
│__internal_decompress_strin│                                                                                                                                                 
│           g(#0)           │                                                                                                                                                 
│__internal_decompress_integ│                                                                                                

'115ms'

## a) Enumérer les ordres de jointure
Combien d'ordre de jointures sont possibles ?

<!-- On a 6! = 720 ordres de jointures possibles -->
on peut combiner jusqu'à (6 * (6 - 1) ) / 2  - 8 = 7

ON a pris les combinaisons possible et on soustrait les couples de tables qu'on ne peut pas joindre notament part et nation par exemple

## b) Requête optimale
 Re-*écrire* cette requête pour qu'elle s'exécute le plus rapidement possible.

In [26]:
query_6_tables_2 = """
SELECT
    n.n_name AS nation,
    EXTRACT(year FROM o.o_orderdate) AS o_year,
    SUM(l.l_extendedprice * (1 - l.l_discount) - ps.ps_supplycost * l.l_quantity) AS sum_profit
FROM
    part p
    JOIN lineitem l ON p.p_partkey = l.l_partkey
    JOIN partsupp ps ON l.l_partkey = ps.ps_partkey AND l.l_suppkey = ps.ps_suppkey
    JOIN orders o ON l.l_orderkey = o.o_orderkey
    JOIN supplier s ON l.l_suppkey = s.s_suppkey
    JOIN nation n ON s.s_nationkey = n.n_nationkey
WHERE
    p.p_name LIKE '%green%'
GROUP BY
    n.n_name,
    EXTRACT(year FROM o.o_orderdate)
ORDER BY
    n.n_name,
    o_year DESC;
"""

# Exécuter la requête
explain_query(query_6_tables_2)
run_query(query_6_tables_2)

┌───────────────────────────┐                                                                                                                                                 
│         PROJECTION        │                                                                                                                                                 
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                                                                                                                                                 
│__internal_decompress_strin│                                                                                                                                                 
│           g(#0)           │                                                                                                                                                 
│__internal_decompress_integ│                                                                                                

'73ms'